In [119]:
!pip install lancedb

You should consider upgrading via the '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper/env/bin/python3 -m pip install --upgrade pip' command.


In [120]:
!pip install tantivy

You should consider upgrading via the '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper/env/bin/python3 -m pip install --upgrade pip' command.


### Required libraries

In [121]:
import lancedb
import pandas as pd
import pyarrow as pa
import sys,os
module_path = os.path.abspath(os.getcwd() + '../../')
sys.path.append(module_path)
print(sys.path)
os.getcwd()
from src.constants import LANCEDB_DICT, EMBEDDINGS_DICT
from sentence_transformers import SentenceTransformer
from lancedb.embeddings import get_registry
from lancedb.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from lancedb.pydantic import LanceModel, Vector

['/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper/env/lib/python3.9/site-packages', '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper']


In [122]:
print(LANCEDB_DICT.get('uri'))

cloud_service_knowledge_base/vector_db


### Create a Database class which takes care of vector database functions

In [123]:

class lanceDB:
    def __init__(self, uri):
        self.uri = uri
        self.db = lancedb.connect(uri)

    def create_table(self, table_name, schema=None, mode="overwrite"):
        print(f"Creating table: {table_name}")
        print(f"mode is {mode}")
        return self.db.create_table(table_name, schema=schema, mode=mode)

    def insert(self, table_name, data):
        self.db.add(table_name, data)

    def query(self, query):
        return self.db.query(query)

    def close(self):
        self.db.close()
        self.async_db.close()
    def table_names(self):
        return self.db.table_names()


### Initiate LanceDB Connection

In [124]:
db = lanceDB(uri=LANCEDB_DICT.get('uri'))
print(db.table_names())


['vector_db']


### Locally Load the sentence transformer model

In [125]:
model_name = EMBEDDINGS_DICT.get('model_name')
device = EMBEDDINGS_DICT.get('device')
model = SentenceTransformer(model_name, device=device)

/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper/env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Verfiy the embeddings

In [126]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
embeddings[0]


array([ 6.76568672e-02,  6.34958893e-02,  4.87131067e-02,  7.93049783e-02,
        3.74480560e-02,  2.65278690e-03,  3.93749177e-02, -7.09835812e-03,
        5.93614504e-02,  3.15369740e-02,  6.00980707e-02, -5.29051721e-02,
        4.06068228e-02, -2.59308200e-02,  2.98427045e-02,  1.12690218e-03,
        7.35148787e-02, -5.03818206e-02, -1.22386619e-01,  2.37028264e-02,
        2.97266264e-02,  4.24768180e-02,  2.56337617e-02,  1.99509831e-03,
       -5.69191240e-02, -2.71598697e-02, -3.29036377e-02,  6.60248548e-02,
        1.19007245e-01, -4.58791889e-02, -7.26215094e-02, -3.25840972e-02,
        5.23412935e-02,  4.50552851e-02,  8.25296901e-03,  3.67024280e-02,
       -1.39415357e-02,  6.53919131e-02, -2.64271721e-02,  2.06365905e-04,
       -1.36643983e-02, -3.62809487e-02, -1.95043124e-02, -2.89738476e-02,
        3.94270532e-02, -8.84091482e-02,  2.62422091e-03,  1.36713963e-02,
        4.83062230e-02, -3.11565585e-02, -1.17329203e-01, -5.11690043e-02,
       -8.85288417e-02, -

In [127]:
# dimensions of the vector embeddings
len(embeddings[0])

384

In [128]:
print(EMBEDDINGS_DICT.get('vector_dims')) # setting this to default under constants

384


### Load LanceDB embedder

In [129]:
# calling the lanceDB embedding functions in the registry
# Using `sentence-transformers` as the model registry
model_registry = get_registry().get("sentence-transformers")
model = model_registry.create(name=model_name, device=device)

### Define the schema for the table

In [130]:
class CloudServiceInestionDataModel(LanceModel):
    description: str = model.SourceField()
    vector_description: Vector(dim=model.ndims()) = model.VectorField()
    cloud_provider: str
    service: str
    category: str =  model.SourceField(),
    id: str
    class Config:
        arbitrary_types_allowed = True

#### Read the dataset as a Pandas Dataframe

In [131]:
df = pd.read_csv('../data/Cloud_Provider_Services_Hashed.csv')

In [132]:
df.tail()

,cloud_provider,category,service,description,id
1297,oci,Network,Network,Network services,764bbda4
1298,oci,Network,Load balancers,Load balancer service,b8c4e52d
1299,oci,Network,Virtual cloud networks,Virtual cloud networks,915f0790
1300,oci,Network,Dns management,DNS management service,fe08c585
1301,oci,Network,Fast connect,Fast connect service,3c3f1427


#### Create a list of documents to be added to LanceDB table

In [133]:
data = df.apply(
    lambda row: {
        "category": row["category"],
        "cloud_provider": row["cloud_provider"],
        "service": row["service"],
        "description": row["description"],
        "id": row["id"],
    },
    axis=1,
).values.tolist()

In [134]:
data[0]

{'category': 'Analytics',
 'cloud_provider': 'aws',
 'service': 'Amazon Athena',
 'description': 'An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL.',
 'id': '95825a76'}

#### Create a lanceDB table with the provided schema model

In [135]:
table = db.create_table("vector_db", schema=CloudServiceInestionDataModel, mode="overwrite")

Creating table: vector_db
mode is overwrite


#### Add the data to the table

In [136]:
table.add(data)

### Full-text search 
[LanceDB - full Text Search documentation](https://lancedb.github.io/lancedb/fts/)

In [137]:
table.create_fts_index("description", replace=True)

In [138]:
#query = "An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL."
query = "query service that makes it easy to analyze data"
fts_actual = table.search(query, query_type="fts").limit(10).to_pandas()

In [139]:
fts_actual.head(10)

,description,vector_description,cloud_provider,service,category,id,score
0,An interactive query service that makes it eas...,"[-0.036750447, -0.038994916, -0.09674644, 0.01...",aws,Amazon Athena,Analytics,95825a76,17.903425
1,A web service that makes it easy to process la...,"[-0.10146084, 0.0067654448, -0.056411035, 0.00...",aws,Amazon Elastic MapReduce,Analytics,1f41d3e8,13.446853
2,A service that makes it easy to add image and ...,"[-0.059520993, -0.046984702, -0.081365086, -0....",aws,Amazon Rekognition,Artificial Intelligence,8191d857,12.409848
3,A fully managed service that makes it easy to ...,"[-0.057634976, -0.030433707, -0.089878656, -0....",aws,Amazon Managed Streaming for Apache Kafka,Compute,21c3010a,12.025981
4,A machine learning service that makes it easy ...,"[-0.084044114, -0.08540694, 0.028245663, 0.045...",aws,Amazon Personalize,Artificial Intelligence,994288f3,10.932827
5,A fast cloud-powered business intelligence ser...,"[-0.029455226, -0.0048628934, -0.032985747, 0....",aws,Amazon QuickSight,Analytics,021db62d,10.621030
6,A fully managed extract transform and load (ET...,"[-0.111309126, 0.046062496, -0.031067966, 0.02...",aws,AWS Glue,Analytics,f1506165,10.530128
7,A fully managed service that makes it easy to ...,"[-0.024797615, -0.012064842, 0.019883556, 0.07...",aws,AWS IoT Analytics,Analytics,67a4d595,10.530128
8,A service that makes it easy for developers of...,"[-0.08342339, -0.06815262, 0.007652386, -0.001...",aws,Amazon Machine Learning,Artificial Intelligence,ce5e7569,10.489511
9,A fully managed service that makes it easy for...,"[-0.06523381, 0.012705022, -0.07271577, 0.0424...",aws,Amazon API Gateway,Compute,c4a5a2c7,10.080750


### Vector Search
[Vector Arithmetic with LanceDB: An Intro to Vector Embeddings](https://blog.lancedb.com/vector-arithmetic-with-lancedb-an-intro-to-vector-embeddings/)

In [140]:
query = "query that makes it easy to analyze S3"
vector_actual = table.search(query, query_type="vector").limit(10).to_pandas()

In [141]:
vector_actual.head(5)

## Expected answer: `Amazon Athena` which is a query service that makes it easy to analyze data in Amazon S3 using standard SQL.

,description,vector_description,cloud_provider,service,category,id,_distance
0,An interactive query service that makes it eas...,"[-0.036750447, -0.038994916, -0.09674644, 0.01...",aws,Amazon Athena,Analytics,95825a76,0.426209
1,Discovery service for data exploration.,"[-0.087175965, -0.023679473, -0.023767654, -0....",ibmcloud,paas_svc_plan_feat_discovery,Service,8b9590fe,1.133843
2,Offers log data collection and analysis.,"[0.0029415176, -0.009669837, -0.03879726, 0.01...",azure,Log Analytics,Analytics,8b8ee64c,1.231552
3,Retrieve and Rank provides tools for building ...,"[-0.046784457, -0.02686176, -0.046415992, -0.0...",ibmcloud,paas_svc_plan_retrieve_and_rank,Service,53037f6a,1.233768
4,Simple query service,"[-0.050498135, -0.024219876, -0.06297479, 0.05...",ibmcloud,paas_svc_plan_tinyqueries,Service,b887cb90,1.240544


### Hybrid Search 

[Example for Hybrid Search from LanceDB github](https://github.com/lancedb/vectordb-recipes/blob/main/examples/Inbuilt-Hybrid-Search/Inbuilt_Hybrid_Search_with_LanceDB.ipynb)

In [142]:
from lancedb.rerankers import LinearCombinationReranker

reranker = LinearCombinationReranker(
    weight=0.8
)  # Weight = 0 Means pure Text Search (BM-25) and 1 means pure Sementic (Vector) Search
query = "query that makes it easy to analyze data"
table.search(
    query,
    query_type="hybrid",
    vector_column_name="vector_description",
).rerank(reranker=reranker).limit(5).to_pandas()

,description,vector_description,cloud_provider,service,category,id,_relevance_score
0,Offers log data collection and analysis.,"[0.0029415176, -0.009669837, -0.03879726, 0.01...",azure,Log Analytics,Analytics,8b8ee64c,0.800000
1,A web service that makes it easy to process la...,"[-0.10146084, 0.0067654448, -0.056411035, 0.00...",aws,Amazon Elastic MapReduce,Analytics,1f41d3e8,0.282639
2,An interactive query service that makes it eas...,"[-0.036750447, -0.038994916, -0.09674644, 0.01...",aws,Amazon Athena,Analytics,95825a76,0.200000
3,Enables you to process and analyze streaming d...,"[0.012544875, -0.029368753, -0.09186228, -0.02...",aws,Amazon Kinesis Analytics,Analytics,bf62df4f,0.178171
4,A service that makes it easy to add image and ...,"[-0.059520993, -0.046984702, -0.081365086, -0....",aws,Amazon Rekognition,Artificial Intelligence,8191d857,0.166611


In [143]:
!pip freeze list

altair==5.4.1
annotated-types==0.7.0
anyio==4.4.0
appnope==0.1.4
asttokens==2.4.1
attrs==24.2.0
blinker==1.8.2
cachetools==5.5.0
certifi==2024.8.30
charset-normalizer==3.3.2
click==8.1.7
comm==0.2.2
debugpy==1.8.5
decorator==5.1.1
deprecation==2.1.0
distro==1.9.0
elastic-transport==8.15.0
elasticsearch==8.15.0
exceptiongroup==1.2.2
executing==2.1.0
filelock==3.15.4
fsspec==2024.9.0
gitdb==4.0.11
GitPython==3.1.43
groq==0.11.0
h11==0.14.0
httpcore==1.0.5
httpx==0.27.2
huggingface-hub==0.24.6
idna==3.8
importlib_metadata==8.4.0
ipykernel==6.29.5
ipython==8.18.1
jedi==0.19.1
Jinja2==3.1.4
jiter==0.5.0
joblib==1.4.2
jsonschema==4.23.0
jsonschema-specifications==2023.12.1
jupyter_client==8.6.2
jupyter_core==5.7.2
lancedb==0.12.0
markdown-it-py==3.0.0
MarkupSafe==2.1.5
matplotlib-inline==0.1.7
mdurl==0.1.2
mpmath==1.3.0
narwhals==1.6.2
nest-asyncio==1.6.0
networkx==3.2.1
numpy==1.26.4
openai==1.43.1
overrides==7.7.0
packaging==24.1
pandas==2.2.2
parso==0.8.4
pexpect==4.9.0
pillow==10.4.0
pla